In [2]:
import praw
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env')

True

In [3]:
# Access and use the environment variables
SECRET_KEY = os.getenv("SECRET_KEY")
USER_AGENT = os.getenv("USER_AGENT")
CLIENT_ID = os.getenv("CLIENT_ID")

In [4]:
#Initializing reddit instance

reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=SECRET_KEY,
    user_agent=USER_AGENT,
)

print(reddit.read_only)

True


In [5]:
redditPost = reddit.subreddit("askreddit")
top_posts = redditPost.top(limit=5, time_filter="day")

In [6]:
def getCommentForPost(comments):
    commentList = []

    for comment in comments:
        if ( len(commentList) > 5 ):
            break

        if ( len(comment.body) < 10 or len(comment.body) > 200 ):
            continue
        
        comment_dic = {
            'id': comment.id,
            'text': comment.body
        }

        commentList.append(comment_dic)

    return commentList


In [7]:
# Convert individual posts to dictionaries
post_list = []
for post in top_posts:
    if post.over_18:
        continue

    post_dict = {
        'title': post.title,
        'score': post.score,
        'url': post.url,
        'id': post.id,
        'comments': getCommentForPost(post.comments)
        # Include other desired fields here
    }
    post_list.append(post_dict)

# Convert the list of posts to JSON format

In [8]:
print(post_list)

[{'title': 'What’s the best example of “it takes 20 years to build a reputation and five minutes to ruin it” that you know of?', 'score': 10265, 'url': 'https://www.reddit.com/r/AskReddit/comments/14tfr7k/whats_the_best_example_of_it_takes_20_years_to/', 'id': '14tfr7k', 'comments': [{'id': 'jr35iwa', 'text': "A minister in my hometown died of autoerotic asphyxiation.  His congretation didn't even hold a service for him.  His family disappeared and within a week it was like he had never existed."}, {'id': 'jr3ytmz', 'text': 'Ellen DeGeneres.  She had a reputation for years of being kind and down to earth. Until the accusations started coming out that she was a real dick to a lot of people and not kind like she portrayed.'}, {'id': 'jr2477l', 'text': 'The backlash on Will Smith for the slap'}, {'id': 'jr2tkns', 'text': 'Ashlee Simpson and the lip synching, jig dancing debacle.'}, {'id': 'jr274tc', 'text': 'I know this has been said before, but the utterly embarrassing Fox interview betw

In [9]:
for post in post_list:
    print(post)

{'title': 'What’s the best example of “it takes 20 years to build a reputation and five minutes to ruin it” that you know of?', 'score': 10265, 'url': 'https://www.reddit.com/r/AskReddit/comments/14tfr7k/whats_the_best_example_of_it_takes_20_years_to/', 'id': '14tfr7k', 'comments': [{'id': 'jr35iwa', 'text': "A minister in my hometown died of autoerotic asphyxiation.  His congretation didn't even hold a service for him.  His family disappeared and within a week it was like he had never existed."}, {'id': 'jr3ytmz', 'text': 'Ellen DeGeneres.  She had a reputation for years of being kind and down to earth. Until the accusations started coming out that she was a real dick to a lot of people and not kind like she portrayed.'}, {'id': 'jr2477l', 'text': 'The backlash on Will Smith for the slap'}, {'id': 'jr2tkns', 'text': 'Ashlee Simpson and the lip synching, jig dancing debacle.'}, {'id': 'jr274tc', 'text': 'I know this has been said before, but the utterly embarrassing Fox interview betwe

In [10]:
for i, post in enumerate(post_list):
    print(post['title'])

What’s the best example of “it takes 20 years to build a reputation and five minutes to ruin it” that you know of?
What's one food that you cannot understand why people enjoy?
What’s a profession that’s occupied exclusively by terrible people?
What’s a tell tale sign someone is bullshitting you?
My cousins from Poland are visiting the U.S. and want to watch an “American Film”, what movies would you recommend?


In [11]:
from controllers.voice_over import createVoiceOver, createVoiceOverPost

for i, post in enumerate(post_list):
    createVoiceOverPost(i, post['title'])
    
    for comment in post['comments']:
        createVoiceOver(comment['id'], comment['text'], i)


In [12]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from constants import SS_HEIGHT, SS_WIDTH, screenshotDir
import os

screenWidth = SS_WIDTH
screenHeight = SS_HEIGHT

def takingSS(driver, wait, target_id, fileName):
    print('target_id', target_id)
    element = wait.until(EC.presence_of_element_located((By.ID, target_id)))
    
    try:
        closeId = 'close'
        ele = driver.find_element(By.ID, closeId)
        ele.click()
        print("Element found and clicked!")
    except NoSuchElementException:
        print("Element not found in the DOM.")

    driver.execute_script("window.focus();")
    ss = f'{fileName}.png'
    fp = open(ss, "wb")
    fp.write(element.screenshot_as_png)
    fp.close()

def initiateDriver(post):
    options = webdriver.ChromeOptions()
    options.enable_mobile = False
    options.add_argument('-headless')
    driver = webdriver.Chrome(options=options)
    url = post['url']
    driver.get(url)
    
    wait = WebDriverWait(driver, 10)
    driver.set_window_size(width=screenWidth, height=screenHeight)

    return driver, wait

for i, post in enumerate(post_list):
    driver, wait = initiateDriver(post)
    fileName = f'screenshots/post-{i}'
    os.mkdir(fileName)
    prefix = f'{fileName}/main'
    post_id = f"t3_{post['id']}"
    takingSS(driver, wait, post_id, prefix)
    driver.quit()

target_id t3_14tfr7k
Element not found in the DOM.
target_id t3_14tcdaj
Element not found in the DOM.
target_id t3_14tc6zu
Element not found in the DOM.
target_id t3_14tnll2
Element not found in the DOM.
target_id t3_14tqsz3
Element not found in the DOM.


In [13]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

def makeScreenShotsForComments(text, path, font_size = 14):
    font_color = (0, 0, 0)  # RGB color for the font
    font_path = "font\Roboto-Black.ttf"  # Replace with the path to your desired font file

    # Create a new image with a white background
    background_color = (255, 255, 255)  # RGB color for the background
    width = 400
    padding = 10  # Padding around the text
    line_spacing = 4  # Spacing between lines of text

    font = ImageFont.truetype(font_path, font_size)
    text_lines = textwrap.wrap(text, width=60)  # Adjust the width parameter as desired

    # Calculate the height of the image based on the text length
    line_height = font.getsize("Sample")[1]
    text_height = (line_height + line_spacing) * len(text_lines)
    height = text_height + (padding * 2)

    image = Image.new("RGB", (width, height), background_color)
    draw = ImageDraw.Draw(image)

    # Calculate the starting position for the text
    text_x = padding
    text_y = padding

    # Draw the text onto the image
    for line in text_lines:
        line_width, line_height = font.getsize(line)
        draw.text((text_x, text_y), line, font=font, fill=font_color)
        text_y += line_height + line_spacing

    image.save(f"{path}.png")
    

In [14]:
for i, post in enumerate(post_list):
    
    for comment in post['comments']:
        path = f"screenshots/post-{i}/comment-{comment['id']}"
        makeScreenShotsForComments(comment['text'], path)


In [15]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def loopingBgVideo(duration, path):
    original_clip = VideoFileClip(path)
    subclip = original_clip.subclip(0, 2)
    looped_clip = concatenate_videoclips([subclip] * (duration // subclip.duration))
    return looped_clip


In [19]:
from moviepy.editor import ImageClip, AudioFileClip, VideoFileClip, CompositeVideoClip
import math

checked_loop_video = None

def createVideoClip(ssFile, voiceFile):
    audioClip = AudioFileClip(voiceFile)
    imageClip = ImageClip(ssFile).set_duration(audioClip.duration)
    videoClip = imageClip.set_audio(audioClip)
    return videoClip

for i, post in enumerate(post_list):
    clips = []
    mainPostSS = f"screenshots/post-{i}/main.png"
    mainPostAudio = f"VoiceOvers/post-{i}/main.mp3"
    mainPostClip = createVideoClip(mainPostSS, mainPostAudio)

    clips.append(mainPostClip)

    start_time = mainPostClip.duration  # Set the start time for the next clip

    for comment in post['comments']:
        pathSS = f"screenshots/post-{i}/comment-{comment['id']}.png"
        pathAudio = f"VoiceOvers/post-{i}/comment-{comment['id']}.mp3"
        commentClip = createVideoClip(pathSS, pathAudio)
        commentClip = commentClip.set_start(start_time)  # Set the start time for the comment clip
        clips.append(commentClip)

        start_time += commentClip.duration  # Update the start time for the next clip

    # bgIndex = random.randint(1, 4)
    bgVideoPath = f"sampleBgVideo/4.mp4"
    bgClip = None
    if checked_loop_video == None:
       bgClip = loopingBgVideo(math.ceil(start_time), bgVideoPath)
       checked_loop_video = True

    # Calculate the center position of the background clip
    bg_center_x = bgClip.w / 2
    bg_center_y = bgClip.h / 2

    # Adjust the position of each clip to center them on the background clip
    for clip in clips:
        clip_center_x = clip.w / 2
        clip_center_y = clip.h / 2
        clip = clip.set_position((bg_center_x - clip_center_x, bg_center_y - clip_center_y))

    finalVideo = CompositeVideoClip(clips=[bgClip] + clips)

    # outputFile = f"finalVideo/reel-{i}.mp4"
    outputFile = 'test-output.mp4'
    finalVideo.write_videofile(outputFile, codec='mpeg4', threads=12, bitrate='8000k')

    break


Moviepy - Building video test-output.mp4.
MoviePy - Writing audio in test-outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test-output.mp4



Moviepy - Done !
Moviepy - video ready test-output.mp4


In [20]:
from moviepy.editor import ImageClip, AudioFileClip, VideoFileClip, CompositeVideoClip
import math

checked_loop_video = None

def createVideoClip(ssFile, voiceFile):
    audioClip = AudioFileClip(voiceFile)
    imageClip = ImageClip(ssFile).set_duration(audioClip.duration)
    videoClip = imageClip.set_audio(audioClip)
    return videoClip

for i, post in enumerate(post_list):
    clips = []
    mainPostSS = f"screenshots/post-{i}/main.png"
    mainPostAudio = f"VoiceOvers/post-{i}/main.mp3"
    mainPostClip = createVideoClip(mainPostSS, mainPostAudio)

    clips.append(mainPostClip)

    start_time = mainPostClip.duration  # Set the start time for the next clip

    for comment in post['comments']:
        pathSS = f"screenshots/post-{i}/comment-{comment['id']}.png"
        pathAudio = f"VoiceOvers/post-{i}/comment-{comment['id']}.mp3"
        commentClip = createVideoClip(pathSS, pathAudio)
        commentClip = commentClip.set_start(start_time)  # Set the start time for the comment clip
        clips.append(commentClip)

        start_time += commentClip.duration  # Update the start time for the next clip

    # bgIndex = random.randint(1, 4)
    bgVideoPath = f"sampleBgVideo/4.mp4"
    bgClip = None
    if checked_loop_video is None:
        bgClip = loopingBgVideo(math.ceil(start_time), bgVideoPath)
        checked_loop_video = True

    # Calculate the center position of the background clip
    bg_center_x = bgClip.w / 2
    bg_center_y = bgClip.h / 2

    # Adjust the position of each clip to center them on the background clip
    for idx, clip in enumerate(clips):
        clip_center_x = clip.w / 2
        clip_center_y = clip.h / 2
        clips[idx] = clip.set_position((bg_center_x - clip_center_x, bg_center_y - clip_center_y))

    finalVideo = CompositeVideoClip(clips=[bgClip] + clips)

    # outputFile = f"finalVideo/reel-{i}.mp4"
    outputFile = 'test-output.mp4'
    finalVideo.write_videofile(outputFile, codec='mpeg4', threads=12, bitrate='8000k')

    break
from moviepy.editor import ImageClip, AudioFileClip, VideoFileClip, CompositeVideoClip
import math

checked_loop_video = None

def createVideoClip(ssFile, voiceFile):
    audioClip = AudioFileClip(voiceFile)
    imageClip = ImageClip(ssFile).set_duration(audioClip.duration)
    videoClip = imageClip.set_audio(audioClip)
    return videoClip

for i, post in enumerate(post_list):
    clips = []
    mainPostSS = f"screenshots/post-{i}/main.png"
    mainPostAudio = f"VoiceOvers/post-{i}/main.mp3"
    mainPostClip = createVideoClip(mainPostSS, mainPostAudio)

    clips.append(mainPostClip)

    start_time = mainPostClip.duration  # Set the start time for the next clip

    for comment in post['comments']:
        pathSS = f"screenshots/post-{i}/comment-{comment['id']}.png"
        pathAudio = f"VoiceOvers/post-{i}/comment-{comment['id']}.mp3"
        commentClip = createVideoClip(pathSS, pathAudio)
        commentClip = commentClip.set_start(start_time)  # Set the start time for the comment clip
        clips.append(commentClip)

        start_time += commentClip.duration  # Update the start time for the next clip

    # bgIndex = random.randint(1, 4)
    bgVideoPath = f"sampleBgVideo/4.mp4"
    bgClip = None
    if checked_loop_video is None:
        bgClip = loopingBgVideo(math.ceil(start_time), bgVideoPath)
        checked_loop_video = True

    # Calculate the center position of the background clip
    bg_center_x = bgClip.w / 2
    bg_center_y = bgClip.h / 2

    # Adjust the position of each clip to center them on the background clip
    for idx, clip in enumerate(clips):
        clip_center_x = clip.w / 2
        clip_center_y = clip.h / 2
        clips[idx] = clip.set_position((bg_center_x - clip_center_x, bg_center_y - clip_center_y))

    finalVideo = CompositeVideoClip(clips=[bgClip] + clips)

    # outputFile = f"finalVideo/reel-{i}.mp4"
    outputFile = 'test-output.mp4'
    finalVideo.write_videofile(outputFile, codec='mpeg4', threads=12, bitrate='8000k')

    break


Moviepy - Building video test-output.mp4.
MoviePy - Writing audio in test-outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test-output.mp4



Moviepy - Done !
Moviepy - video ready test-output.mp4
Moviepy - Building video test-output.mp4.
MoviePy - Writing audio in test-outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test-output.mp4



Moviepy - Done !
Moviepy - video ready test-output.mp4
